# Exercises on 3D animations with matrix transformations

## Exercise 5.1
Write a function `infer_matrix(n, transformation)` that takes a dimension and a function that is a vector transformation assumed to be linear and returns an nxn square matrix that represents the linear transformation.

Note that the given `transformation` **has** to be linear, otherwise, the returned matrix will not represent the transformation.

We know that to completely specify a linear transformation, it is enough to specify how it transforms the vectors of the standard basis.

Thus, what we have to do is is apply the given transformation to the standard basis of the given dimension, collect the results in columns, and return that matrix.

Let's start with a supporting function that will return the standard basis vectors.

For the standard basis vectors, we need to come up with a collection of tuples on which the coordinate 1 is shifted in position for every vector.

```
n = 2:
    e1 = (1, 0)
    e2 = (0, 1)
    standard_basis = [e1, e2]

n = 3:
    e1 = (1, 0, 0)
    e2 = (0, 1, 0)
    e3 = (0, 0, 1)
    standard_basis = [e1, e2, e3]

n = 4:
    e1 = (1, 0, 0, 0)
    e2 = (0, 1, 0, 0)
    e3 = (0, 0, 1, 0)
    e4 = (0, 0, 0, 1)
    standard_basis = [e1, e2, e3, e4]
```

Now, we're beginning to see that we will need a couple of nested iterations: one over the number of dimensions given (number of standard basis vectors) and an inner one to build the elements of each of the vectors.

Let's start with a *procedural*, non-Pythonic way:

In [3]:
def standard_basis(dimension):
    standard_basis = []
    for i in range(0, dimension):
        e = []
        for j in range(0, dimension):
            if (i == j):
                e.append(1)
            else:
                e.append(0)
        standard_basis.append(e)
    return standard_basis

print(standard_basis(2))
print(standard_basis(3))
print(standard_basis(4))

[[1, 0], [0, 1]]
[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]


That solution more or less works, but we are only using lists instead of tuples, and it cries for a more succinct implementation.

Let's give it a second try with the hint that list and tuple comprehensions in Python allows for using if conditions in their definition:

In [7]:
def standard_basis(dimension):
    standard_basis = [
        tuple(1 if i == j else 0 for j in range(0, dimension)) 
        for i in range(0, dimension)
        ]
    return standard_basis

print(standard_basis(2))
print(standard_basis(3))
print(standard_basis(4))

[(1, 0), (0, 1)]
[(1, 0, 0), (0, 1, 0), (0, 0, 1)]
[(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)]


Note that for readability purposes, the author recommends defining a support function to return the standard basis vector, but either way is fine.



In [8]:
def standard_basis(dimension):
    def standard_basis_vector(i):
        return tuple(1 if i == j else 0 for j in range(0, dimension))
    standard_basis = [standard_basis_vector(i) for i in range(0, dimension)]
    return standard_basis

print(standard_basis(2))
print(standard_basis(3))
print(standard_basis(4))

[(1, 0), (0, 1)]
[(1, 0, 0), (0, 1, 0), (0, 0, 1)]
[(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)]


In [ ]:
Now, we need to define our main function:

In [9]:
def infer_matrix(n, transformation):
    transformations_on_standard_basis = [transformation(ei) for ei in standard_basis(n)]
    return tuple(zip(*transformations_on_standard_basis))

Note that we had to use `zip(*matrix)` to convert the matrix rows into columns.

Let's validate it with a transformation like `rotate_z_by(pi / 2)`:

In [12]:
from my_transformations import rotate_z_by
from math import pi

infer_matrix(3, rotate_z_by(pi / 2))

((6.123233995736766e-17, -1.0, 0.0),
 (1.0, 1.2246467991473532e-16, 0.0),
 (0, 0, 1))

We can validate it also visually, by using this function to rotate our teapot:

In [19]:
from my_teapot import load_triangles
from my_draw_model import draw_model
from math import sin, cos
from my_transformations import rotate_z_by, polygon_map
from my_matrices import multiply_matrix_vector
from math import pi

def standard_basis(dimension):
    def standard_basis_vector(i):
        return tuple(1 if i == j else 0 for j in range(0, dimension))
    standard_basis = [standard_basis_vector(i) for i in range(0, dimension)]
    return standard_basis

def infer_matrix(n, transformation):
    transformations_on_standard_basis = [transformation(ei) for ei in standard_basis(n)]
    return tuple(zip(*transformations_on_standard_basis))

rotation_matrix = infer_matrix(3, rotate_z_by(pi / 2))

def matrix_rotate_z_by(angle_radians):
    matrix_transformation = infer_matrix(3, rotate_z_by(angle_radians))
    def new_function(v):
        return multiply_matrix_vector(matrix_transformation, v)
    return new_function

draw_model(polygon_map(matrix_rotate_z_by(pi / 2), load_triangles()))

SystemExit: 0

In [ ]:
Note that as this rotation matrix does not depend on time, the teapot is not animated, but rather rotated once.

# Exercise 5.2

What is the result of the following product?

$
\begin{pmatrix}
1.3 & -0.7 \\
6.5 & 3.2
\end{pmatrix}
\begin{pmatrix}
-2.5 \\
0.3
\end{pmatrix}
$

The result will be a column vector with dimension 2, with $ a_{11} = (1.3, -0.7) \cdot (-2.5, 0.3) $ and $ a_{21} = (6.5, 3.2) \cdot (-2.5, 0.3) $

$
\begin{pmatrix}
1.3 & -0.7 \\
6.5 & 3.2
\end{pmatrix}
\begin{pmatrix}
-2.5 \\
0.3
\end{pmatrix} = \begin{pmatrix}
-3.46 \\
-15.29
\end{pmatrix}
$


Let's validate it with our Python functions:

In [21]:
from my_matrices import multiply_matrix_vector

m = (
    (1.3, -0.7),
    (6.5, 3.2)
    )
v = (-2.5, 0.3)

print(multiply_matrix_vector(m, v))

(-3.46, -15.29)


# Exercise 5.3

Write a `random_matrix`